# Скачивание тайликов яндекс карт на локальное хранилище

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import sys
sys.path.append('../utils')

from tile_downloader import download_map_tile

**берем z = 17, размер тайлика на широте Египта - примерно 200 x 200 метров**

In [ ]:
zoom = 17

## Считываем координаты и скачиваем positive samples

In [ ]:
positives_df = pd.read_csv('../coordinates_data/Eastern_desert_archaeological_structures.csv', delimiter=',')

In [ ]:
errors = []
limit = len(positives_df)
for i in tqdm(range(limit)):
    id_, name, _, lon, lat, _, _, _, _, _, _, _ = positives_df.iloc[i]
    try:
        img = download_map_tile(lon=lon, lat=lat, z=zoom, pixel_size=256)
        img.save(f'../yandexmaps_data/positive_samples/{name}.png')
    except Exception as e:
        errors.append(name)
print(f'{len(errors)=}')

## Генерируем координаты и скачиваем negative samples

In [ ]:
lon_coords = np.array(positives_df['x coordinate'])
lat_coords = np.array(positives_df['y coordinate'])
n = len(lon_coords)

In [ ]:
np.random.seed(42)

errors = []
generated_coords = []

N = 200000
for ind in tqdm(range(N)):

    weights = np.random.random(size=n)
    weights /= np.sum(weights)

    lon = np.dot(lon_coords, weights)
    lat = np.dot(lat_coords, weights)

    generated_coords.append((lon, lat))
    
    try:
        local_file_name = f'../yandexmaps_data/negative_sampels/img{str(ind).zfill(6)}.png'
        img = download_map_tile(lon=lon, lat=lat, z=zoom, pixel_size=256)
        img.save(local_file_name)

    except Exception as e:
        print(e)
        errors.append((ind, lon, lat))
        continue

print(f'{len(errors)=}')

## Скачиваем координаты negative_samples

In [ ]:
with open('../yandexmaps_data/coords.csv', 'w') as f:
    f.write('img_id,lon,lat\n')
    for i, (lon, lat) in enumerate(generated_coords):
        f.write(f'{i},{lon},{lat}\n')